In [3]:
import numpy as np
import random as rd
# Set de datos para la puerta XOR
x_train = np.array([[0,0],[0,1],[1,0],[1,1]], dtype = int)
y_train = np.array([[0],[1],[1],[0]], dtype = int)

In [4]:
def relu(z):
    result = z
    result[z < 0] = 0
    return result
 
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-1 * z))

In [5]:
def predict_n_evaluate(weights_mat, train_inputs, train_outputs):
    predictions = np.zeros(shape=(train_inputs.shape[0]))
    # Loop de todas la entradas
    for sample_id in range(train_inputs.shape[0]):
      # Input
      y = train_inputs[sample_id, :]
      # Capa oculta
      y = np.matmul(y, weights_mat[0])
      # Funcion de activacion
      y = relu(y)
      # Capa de salida
      y = np.matmul(y, weights_mat[1])
      # Funcion de activacion
      y = sigmoid(y)
      predicted_value = np.round(y)
      predictions[sample_id] = predicted_value
    predictions = np.reshape(predictions, (4, 1))
    # Predicciones correctas
    correct_predictions = np.where(predictions == train_outputs)[0].size
    # Calculo de accuracy
    accuracy = (correct_predictions / train_outputs.size) * 100
    return accuracy, predictions

In [6]:
def evaluate_population(weights_mat, data_inputs, data_outputs):
    accuracy = np.empty(shape=(len(weights_mat)))
    for chromosome in range(len(weights_mat)):
        chromosome_mat = weights_mat[chromosome]
        accuracy[chromosome], _ = predict_n_evaluate(chromosome_mat, data_inputs, data_outputs)
    return accuracy

In [7]:
# Convierte el cromosoma de matriz a vector para operaciones del algoritmo genetico
def mat_to_vector(mat_pop_weights):
    pop_weights_vector = []
    for sol_id in range(len(mat_pop_weights)):
        curr_vector = []
        for layer_id in range(len(mat_pop_weights[1])):
            vector_weights = np.reshape(mat_pop_weights[sol_id][layer_id], newshape=(mat_pop_weights[sol_id][layer_id].size))
            curr_vector.extend(vector_weights)
        pop_weights_vector.append(curr_vector)
    return pop_weights_vector
 
# Convierte el cromosoma de vector a matriz para evaluarlo en la red neuronal
def vector_to_mat(vector_pop_weights, mat_pop_weights):
    list_mat_weights = []
    for sol_id in range(len(mat_pop_weights)):
        start = 0
        end = 0
        mat_weights = []
        for layer_id in range(len(mat_pop_weights[1])):
            end = end + mat_pop_weights[sol_id][layer_id].size
            curr_vector = vector_pop_weights[sol_id][start:end]
            mat_layer_weights = np.reshape(curr_vector, newshape=(mat_pop_weights[sol_id][layer_id].shape))
            mat_weights.append(mat_layer_weights)
            start = end
        list_mat_weights.append(mat_weights)
    return list_mat_weights

In [8]:
def select_parents(population, evaluation, num_parents):
    parents = []
    for parent_num in range(num_parents):
        max_evaluation_id = np.where(evaluation == np.max(evaluation))
        max_evaluation_id = max_evaluation_id[0][0]
        parents.append(population[max_evaluation_id])
        evaluation[max_evaluation_id] = -99999999999
    return parents

In [9]:
def uniform_random_crossover(parents, offspring_size):
    offspring = []
    filter_vec_1 = np.random.randint(2, size=(offspring_size[1]))
    filter_vec_2 = filter_vec_1^1
    for pair in range(offspring_size[0]):
        # Id del primer padre
        parent1_id = pair%len(parents)
        # Id del segundo padre
        parent2_id = (pair+1)%len(parents)
        # Creación de la descendencia
        offspring.append((parents[parent1_id] * filter_vec_1 + parents[parent2_id] * filter_vec_2).tolist())
    return offspring

In [10]:
def mutation(offspring, gen_mutation_percent, cro_mutation_percent):
  # Cantidad de genes que mutan en un cromosoma mutante
  num_mutations_gen = np.uint32((gen_mutation_percent*len(offspring[1]))/100)
  # Indices de los genes que mutan en un cromosoma mutante
  mutation_indices_gen = np.array(rd.sample(range(0, len(offspring[1])), num_mutations_gen))
  # Cantidad de cromosomas que mutan en la poblacion
  num_mutations_cro = np.uint32((cro_mutation_percent*len(offspring))/100)
  # Indices de los cromosomas que mutan
  mutation_indices_cro = np.array(rd.sample(range(0, len(offspring)), num_mutations_cro))
  for id_cro in mutation_indices_cro:
    for id_gen in mutation_indices_gen: 
      random_value = rd.uniform(-0.1, 0.1)
      offspring[id_cro][id_gen] = offspring[id_cro][id_gen] + random_value
  return offspring

In [11]:
# Tamaño de la población
sol_per_pop = 16
# Cantidad de padres en la seleccion
num_parents = 8
# Cantidad de generaciones
num_generations = 100
# Porcentaje de genes que mutan en un cromosoma mutante
gen_mutation_percent = 30
# Porcentaje de descendientes que mutan
cro_mutation_percent = 50
# Creación de la población inicial
initial_pop_weights = []
for curr_sol in np.arange(0, sol_per_pop):
  # La capa oculta tiene dos neuronas
  HL_neurons = 2
  input_HL_weights = np.random.uniform(-3, 3, size=(x_train.shape[1], HL_neurons))
  # La capa de salida tiene una neurona
  output_neurons = 1
  HL_output_weights = np.random.uniform(-3, 3, size=(HL_neurons, output_neurons))
  initial_pop_weights.append([input_HL_weights, HL_output_weights])
pop_weights_mat = initial_pop_weights
pop_weights_vector = mat_to_vector(pop_weights_mat)
accuracies = np.empty(shape=(num_generations))

In [12]:
for generation in range(num_generations):
  print("Generación : ", generation)
 
  # ------------------------------------------------------------------------
  # -- 1. EVALUACION DE LA POBLACION                                      --
  # ------------------------------------------------------------------------
 
  # Creamos la matrices de pesos a partir de los vectores. (cromosomas)
  pop_weights_mat = vector_to_mat(pop_weights_vector, pop_weights_mat)
 
  # Evaluamos la poblacion de cromosomas.
  evaluation = evaluate_population(pop_weights_mat, x_train, y_train)
  print("1. Evaluacion de Cromosomas")
  print(evaluation)
 
  # ------------------------------------------------------------------------
  # -- 2. SELECCION DE LOS MEJORES CROMOSOMAS                             --
  # ------------------------------------------------------------------------
 
  parents = select_parents(pop_weights_vector, evaluation.copy(), num_parents)
  print("2. Padres Seleccionados")
  print(parents)
 
  # ------------------------------------------------------------------------
  # -- 3. CRUZA DE LOS CROMOSOMAS SELECCIONADOS                           --
  # ------------------------------------------------------------------------
 
  crossover = uniform_random_crossover(parents, offspring_size=(len(pop_weights_vector)-len(parents), len(pop_weights_vector[0])))
  print("3. Descendencia por Cruza Aleatoria Uniforme")
  print(crossover)
 
  # ------------------------------------------------------------------------
  # -- 4. MUTACION DE LA DESCENDENCIA                                     --
  # ------------------------------------------------------------------------ 
 
  final_offspring = mutation(crossover, gen_mutation_percent, cro_mutation_percent)
  print("4. Descendencia afectada por Mutación")
  print(final_offspring)
 
  # ------------------------------------------------------------------------
  # -- 5. CREACION DE LA NUEVA POBLACION                                  --
  # ------------------------------------------------------------------------
 
  pop_weights_vector = parents + final_offspring
  print("5. Nueva Generación")
  print(pop_weights_vector)

Generación :  0
1. Evaluacion de Cromosomas
[50. 75. 75. 75. 75. 50. 75. 50. 50. 50. 75. 50. 50. 75. 50. 75.]
2. Padres Seleccionados
[[-2.787559956987255, -2.344453706349516, 2.7039264682338278, 0.034420295314072646, 1.0232122314811773, 1.7446513192179758], [-1.421490199588537, 0.3296427776522819, 0.9395958823852597, 0.8933291587364276, 1.3336675644867855, 0.5927258104030466], [0.4271504510941524, -2.1172093619282357, 0.927213848145251, -2.3542761249564697, 2.2121033635746645, -1.332171708983824], [2.4629888605227848, 2.1960301709579477, 1.5459919180003894, 1.4501054239095623, -1.4529671458792353, 1.9679977822443888], [0.9635790458575162, 1.5802075049363733, -1.2416301469359148, 1.1630090576693366, 1.3567964492893916, -0.1248028513345929], [2.5640886908845024, -2.872088658185258, -1.8153209620077861, 2.941752113024286, -2.251838603021366, 0.898464999412552], [0.5713994992580864, -1.5772076277099019, 1.526795932461857, -1.021545380585716, 2.734744814306069, 0.7606630892936597], [-1.465